In [16]:
import rdflib
from rdflib.plugins.sparql import prepareQuery
from tabulate import tabulate

In [17]:
filename = "../../data/02/ABox.ttl"

In [18]:
text1 = '''CQ_2.1
Return the textual descriptions assigned to the risks, the agents of deterioration that classify them, and their types.
'''

query1 = '''
PREFIX hero: <http://purl.org/sirius/ontology/hero/>

SELECT ?risk ?agent_of_deterioration ?risk_type ?risk_summary
WHERE {
    ?risk_assessment a hero:IdentificationDescription ;
                    hero:identifies ?risk .
    ?risk a hero:Risk ;
        hero:isClassifiedByAgent ?agent_of_deterioration ;
        hero:isClassifiedByType ?risk_type .
    OPTIONAL {
        ?risk_assessment hero:hasNote ?risk_summary .
    }
}
'''

In [19]:
text2 = '''CQ_2.2
Return the hazards originating the risk that are related to the agent of deterioration "Physical forces".
'''

query2 = '''
PREFIX hero: <http://purl.org/sirius/ontology/hero/>
PREFIX ti: <http://www.ontologydesignpatterns.org/cp/owl/timeinterval.owl#>
PREFIX tis: <http://ontologydesignpatterns.org/cp/owl/timeindexedsituation.owl#>

SELECT ?risk ?risk_type ?document ?start_date ?end_date
WHERE {
    ?risk_assessment a hero:IdentificationDescription ;
                    hero:isDocumentedBy ?document ;
                    tis:atTime ?time_interval ;
                    hero:identifies ?risk .
    ?time_interval ti:hasIntervalStartDate ?start_date ;
                    ti:hasIntervalEndDate ?end_date .
    ?risk a hero:Risk ;
            hero:isClassifiedByType ?risk_type ;
            hero:isClassifiedByLayer ?layer .
    FILTER (
        ?layer = hero:site ||
        ?layer = hero:region
    )
}
'''

In [20]:
queries = [(text1, query1),
           (text2, query2)
           ]

g = rdflib.ConjunctiveGraph()
g.parse(filename, format="turtle", encoding="utf-8")

for query in queries:
    q = prepareQuery(query[1])
    results = g.query(q)
    print(query[0])
    table = []
    for row in results:
        table.append([row[var] for var in results.vars])
    print(tabulate(table, headers=results.vars, tablefmt="psql"))

CQ_2.1
Return the textual descriptions assigned to the risks, the agents of deterioration that classify them, and their types.

+------------------------------------------------------------+------------------------------------------------------+-------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+
| risk                                                       | agent_of_deterioration                               | risk_type                                       | risk_summary                                                                                                                                                  |
|------------------------------------------------------------+------------------------------------------------------+-------------------------------------------------+-------------------------------------------------